In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot
import keras
import tensorflow as tf
from keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import GridSearchCV
import sklearn
from sklearn.model_selection import RandomizedSearchCV

from keras.wrappers.scikit_learn import KerasClassifier

from skopt import dummy_minimize


from keras.callbacks import ReduceLROnPlateau

from skopt import gp_minimize
from skopt.utils import use_named_args

In [2]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

def load_dataset():
    (trainX, trainY), (testX, testY) = fashion_mnist.load_data()
    trainX = trainX.reshape((trainX.shape[0], 28, 28, 1))
    testX = testX.reshape((testX.shape[0], 28, 28, 1))
    trainY = to_categorical(trainY)
    testY = to_categorical(testY)
    return trainX, trainY, testX, testY

def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm   

Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [3]:
#TEST RUN

trainX, trainY, testX, testY = load_dataset()
trainX, testX = prep_pixels(trainX, testX)



In [4]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.
  Created wheel for databricks-cli: filename=databricks_cli-0.16.4-py3-none-any.whl size=106877 sha256=f6c5234f954f216fcb4ee38a76bec79bb78ec12adfbde73e2d496ec691c3fec7
  Stored in directory: c:\users\cust\appdata\local\pip\cache\wheels\d0\a9\13\b6cc3a08af2b683fd98cafff02c0207cdebb512010ad964607
Successfully built databricks-cli


In [6]:
import optuna
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop
import mlflow
from mlflow.tracking import MlflowClient
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID

In [7]:

def objective(trial):

  with mlflow.start_run()as run:
    model=Sequential()

  #CONVOLUTIONAL LAYER

    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(28, 28, 1)))
  
  #MAX POOLING LAYER
    model.add(MaxPooling2D((2, 2)))

  #FLATTENING THE OUTPUT

    model.add(Flatten())

  # DEFINE A SEARCH SPACE FOR REGULARIZER

    regularizer_type = trial.suggest_categorical('regularizer_type',['l1','l2','l1_l2'])

    mlflow.log_param("regularizer_type", regularizer_type)


    if regularizer_type == 'l1':
      regularizer = regularizers.l1(l1=trial.suggest_loguniform('weight',0.001,0.1))
      

    if regularizer_type =='l2':
      regularizer = regularizers.l2(l2=trial.suggest_loguniform('weight',0.001,0.1))
      

    if regularizer_type =='l1_l2':
      regularizer = regularizers.l1_l2(l1=trial.suggest_loguniform('weight1',0.001,0.1),l2=trial.suggest_loguniform('weight2',0.001,0.1))
      
    
 #DENSE LAYERS
    model.add(Dense(188, activation='relu', kernel_initializer='he_uniform',kernel_regularizer=regularizer))
    model.add(Dense(10, activation='softmax'))


  #VARYING THE OPTIMIZER

    optimizer_name = trial.suggest_categorical('optimizer_name',['Adam','RMSprop'])

    mlflow.log_param("optimizer_name", optimizer_name)


    if optimizer_name == 'Adam':
      optimizer = Adam(learning_rate = trial.suggest_float('learning_rate',0.00001,0.0001))

    if optimizer_name == 'RMSprop':
      optimizer = RMSprop(learning_rate= trial.suggest_float('learning_rate',0.00001,0.0001),momentum=trial.suggest_float('momentum',0.6,0.95))

  #COMPILE THE MODEL
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    #VARY THE BATCH SIZE

    batch_size = trial.suggest_categorical('batch_size',[32,64,128])
    mlflow.log_param("batch_size", batch_size)


  #FIT THE MODEL

    history= model.fit(
        trainX,
        trainY,
        batch_size=batch_size,
        epochs=5,
        verbose=1,
        validation_data=(testX, testY))
    
    
  
    accuracy = history.history['val_accuracy'][-1]

    mlflow.log_metric("accuracy",accuracy)
  
    print('accuracy is ', accuracy)
  
    global best_accuracy
  
    del model

    

    return accuracy

In [ ]:
from optuna.integration.mlflow import MLflowCallback

mlflow.set_experiment('my_experiment')

if __name__ == "__main__":
  mlflc = MLflowCallback(tracking_uri='ml_exp1',metric_name='accuracy')
  study= optuna.create_study(
    direction='maximize',
    study_name='cnn_study',
    load_if_exists=True)
  study.optimize(objective, n_trials=30, callbacks=[mlflc])

<ipython-input-9-db36eaa41467>:6: ExperimentalWarning: MLflowCallback is experimental (supported from v1.4.0). The interface can change in the future.
  mlflc = MLflowCallback(tracking_uri='ml_exp1',metric_name='accuracy')
[I 2022-03-22 23:27:48,213] A new study created in memory with name: cnn_study
2022/03/22 23:27:48 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a

Epoch 1/5
1875/1875 [==============================] - 50s 25ms/step - loss: 9.5254 - accuracy: 0.7947 - val_loss: 4.1626 - val_accuracy: 0.8265
Epoch 2/5
1875/1875 [==============================] - 50s 26ms/step - loss: 2.9266 - accuracy: 0.8324 - val_loss: 2.1394 - val_accuracy: 0.8219
Epoch 3/5
1875/1875 [==============================] - 52s 28ms/step - loss: 1.7313 - accuracy: 0.8301 - val_loss: 1.4555 - val_accuracy: 0.8220
Epoch 4/5
1875/1875 [==============================] - 54s 29ms/step - loss: 1.2784 - accuracy: 0.8278 - val_loss: 1.1700 - val_accuracy: 0.8205
Epoch 5/5
1875/1875 [==============================] - 60s 32ms/step - loss: 1.0649 - accuracy: 0.8288 - val_loss: 1.0315 - val_accuracy: 0.8047


[I 2022-03-22 23:32:18,335] Trial 0 finished with value: 0.8047000169754028 and parameters: {'regularizer_type': 'l1', 'weight': 0.0011301156984240145, 'optimizer_name': 'Adam', 'learning_rate': 2.639184046128422e-05, 'batch_size': 32}. Best is trial 0 with value: 0.8047000169754028.
2022/03/22 23:32:18 INFO mlflow.tracking.fluent: Experiment with name 'cnn_study' does not exist. Creating a new experiment.


accuracy is  0.8047000169754028
Epoch 1/5
938/938 [==============================] - 45s 47ms/step - loss: 9.3714 - accuracy: 0.6810 - val_loss: 1.9546 - val_accuracy: 0.7216
Epoch 2/5
938/938 [==============================] - 45s 48ms/step - loss: 1.8635 - accuracy: 0.7336 - val_loss: 1.8156 - val_accuracy: 0.7373
Epoch 3/5
938/938 [==============================] - 53s 56ms/step - loss: 1.7313 - accuracy: 0.7469 - val_loss: 1.7125 - val_accuracy: 0.7424
Epoch 4/5
938/938 [==============================] - 43s 45ms/step - loss: 1.6576 - accuracy: 0.7554 - val_loss: 1.6329 - val_accuracy: 0.7562
Epoch 5/5
938/938 [==============================] - 37s 39ms/step - loss: 1.6157 - accuracy: 0.7607 - val_loss: 1.5986 - val_accuracy: 0.7615


[I 2022-03-22 23:36:01,559] Trial 1 finished with value: 0.7615000009536743 and parameters: {'regularizer_type': 'l1', 'weight': 0.012893358719873663, 'optimizer_name': 'RMSprop', 'learning_rate': 6.123906213143902e-05, 'momentum': 0.8371379734070274, 'batch_size': 64}. Best is trial 0 with value: 0.8047000169754028.


...

accuracy is  0.5397999882698059
Epoch 1/5
469/469 [==============================] - 42s 88ms/step - loss: 232.3201 - accuracy: 0.4745 - val_loss: 38.9604 - val_accuracy: 0.6141
Epoch 2/5
469/469 [==============================] - 30s 63ms/step - loss: 6.9526 - accuracy: 0.5982 - val_loss: 2.2717 - val_accuracy: 0.5578
Epoch 3/5
469/469 [==============================] - 32s 69ms/step - loss: 2.1922 - accuracy: 0.5716 - val_loss: 2.1271 - val_accuracy: 0.5577
Epoch 4/5
469/469 [==============================] - 40s 84ms/step - loss: 2.0540 - accuracy: 0.5785 - val_loss: 1.9987 - val_accuracy: 0.5896
Epoch 5/5
469/469 [==============================] - 33s 71ms/step - loss: 1.9384 - accuracy: 0.5924 - val_loss: 1.8919 - val_accuracy: 0.5863


[I 2022-03-23 00:02:24,192] Trial 7 finished with value: 0.5863000154495239 and parameters: {'regularizer_type': 'l1_l2', 'weight1': 0.03013434469951901, 'weight2': 0.01784410043608344, 'optimizer_name': 'Adam', 'learning_rate': 5.254992588557392e-05, 'batch_size': 128}. Best is trial 2 with value: 0.8689000010490417.


accuracy is  0.5863000154495239
Epoch 1/5
938/938 [==============================] - 38s 40ms/step - loss: 36.6297 - accuracy: 0.6536 - val_loss: 1.8217 - val_accuracy: 0.6523
Epoch 2/5
938/938 [==============================] - 51s 55ms/step - loss: 1.6539 - accuracy: 0.6912 - val_loss: 1.5596 - val_accuracy: 0.6845
Epoch 3/5
938/938 [==============================] - 41s 43ms/step - loss: 1.4674 - accuracy: 0.7203 - val_loss: 1.4567 - val_accuracy: 0.7041
Epoch 4/5
938/938 [==============================] - 50s 54ms/step - loss: 1.3544 - accuracy: 0.7357 - val_loss: 1.3103 - val_accuracy: 0.7322
Epoch 5/5
938/938 [==============================] - 43s 46ms/step - loss: 1.2779 - accuracy: 0.7435 - val_loss: 1.2703 - val_accuracy: 0.7340


[I 2022-03-23 00:06:07,797] Trial 8 finished with value: 0.734000027179718 and parameters: {'regularizer_type': 'l1', 'weight': 0.015118878129860746, 'optimizer_name': 'Adam', 'learning_rate': 9.174521715770914e-05, 'batch_size': 64}. Best is trial 2 with value: 0.8689000010490417.


accuracy is  0.734000027179718
Epoch 1/5
938/938 [==============================] - 44s 45ms/step - loss: 15.1185 - accuracy: 0.6342 - val_loss: 3.5695 - val_accuracy: 0.7146
Epoch 2/5
329/938 [=========>....................] - ETA: 31s - loss: 3.4765 - accuracy: 0.7140